In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import warnings 
warnings.filterwarnings("ignore")
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!git clone -b feature/add_hf https://github.com/nielsrogge/ImageBind.git
%cd ImageBind

Cloning into 'ImageBind'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 125 (delta 57), reused 39 (delta 39), pack-reused 40 (from 1)
Receiving objects: 100% (125/125), 2.64 MiB | 23.50 MiB/s, done.
Resolving deltas: 100% (59/59), done.
/kaggle/working/ImageBind


In [3]:
!pip install .

Processing /kaggle/working/ImageBind
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/pytorchvideo.git (to revision 28fe037d212663c6a24f373b94cc5d478c8c1a1d) to /tmp/pip-install-hkikix2d/pytorchvideo_a3c8aa874ada40f6bd760c10dd6441dd
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorchvideo.git /tmp/pip-install-hkikix2d/pytorchvideo_a3c8aa874ada40f6bd760c10dd6441dd
  Running command git rev-parse -q --verify 'sha^28fe037d212663c6a24f373b94cc5d478c8c1a1d'
  Running command git fetch -q https://github.com/facebookresearch/pytorchvideo.git 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Running command git checkout -q 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Resolved https://github.com/facebookresearch/pytorchvideo.git to commit 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (

In [4]:
from imagebind import data
import torch
from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType
from imagebind.models.imagebind_model import ImageBindModel

text_list=["A dog.", "A car", "A bird"]
image_paths=[".assets/dog_image.jpg", ".assets/car_image.jpg", ".assets/bird_image.jpg"]
audio_paths=[".assets/dog_audio.wav", ".assets/car_audio.wav", ".assets/bird_audio.wav"]

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = ImageBindModel.from_pretrained("nielsr/imagebind-huge")
model.eval()
model.to(device)


config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.80G [00:00<?, ?B/s]

ImageBindModel(
  (modality_preprocessors): ModuleDict(
    (vision): RGBDTPreprocessor(
      (cls_token): tensor((1, 1, 1280), requires_grad=True)
      
      (rgbt_stem): PatchEmbedGeneric(
        (proj): Sequential(
          (0): PadIm2Video()
          (1): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
        )
      )
      (pos_embedding_helper): SpatioTemporalPosEmbeddingHelper(
        (pos_embed): tensor((1, 257, 1280), requires_grad=True)
        
      )
    )
    (text): TextPreprocessor(
      (pos_embed): tensor((1, 77, 1024), requires_grad=True)
      (mask): tensor((77, 77), requires_grad=False)
      
      (token_embedding): Embedding(49408, 1024)
    )
    (audio): AudioPreprocessor(
      (cls_token): tensor((1, 1, 768), requires_grad=True)
      
      (rgbt_stem): PatchEmbedGeneric(
        (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10), bias=False)
        (norm_layer): LayerNorm((768,), eps=1e-05, elementwise_affine=

In [5]:
import ast
input_df = pd.read_csv('/kaggle/input/business-json/Test (1).csv') 
# Function to extract entity name from the 'entities' column
def extract_entity_name(entities_str):
    # Use ast.literal_eval to safely evaluate the string
    entities_list = ast.literal_eval(entities_str)
    # Extract the 'mention' field from the first entity in the list
    return entities_list[0]['mention'] if entities_list else None

# Apply the function to the 'entities' column
input_df['entity_name'] = input_df['entities'].apply(extract_entity_name)
entity_df = pd.read_csv('/kaggle/input/business-json/Knowledge.csv')
asr_df = pd.read_csv("/kaggle/input/business-json/asr_out.csv")

In [6]:
import os
import torch
from imagebind.models.imagebind_model import ModalityType, ImageBindModel
from imagebind import data
import pandas as pd
from tqdm import tqdm
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Function to process audio files in batches
def process_audio_in_batches(txt, batch_size=16):
    all_embeddings = []
    for i in tqdm(range(0, len(txt), batch_size), desc="Processing Batches"):
        batch_txt = txt[i:i + batch_size]
        embed_inputs = data.load_and_transform_text(batch_txt, device)
        
        with torch.no_grad():
            embeddings = model({
                ModalityType.TEXT: embed_inputs,
            })
        
        all_embeddings.append(embeddings[ModalityType.TEXT])
        torch.cuda.empty_cache()
    
    return torch.cat(all_embeddings, dim = 0)

# # Process entity audio files in batches
entity_embeddings= process_audio_in_batches(entity_df['Knowledge'].tolist())

# Process sentence audio files in batches
input_embeddings  = process_audio_in_batches(asr_df['ASR_Output'].tolist())



Processing Batches: 100%|██████████| 165/165 [00:47<00:00,  3.45it/s]


In [7]:
from sklearn.metrics.pairwise import cosine_similarity
# Initialize counters for ranking accuracy
top1_correct = 0
top5_correct = 0
top10_correct = 0

res_data = []

# Step 3: Calculate cosine similarity and evaluate rankings
for i, sentence in enumerate(input_df['text']):
    # Get the similarity scores for the current sentence with all entities
    similarity_scores = cosine_similarity(input_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    
    # Get the indices of entities sorted by similarity score (descending order)
    ranked_entity_indices = similarity_scores.argsort()[::-1]
    
    # Get the ranked entity names
    ranked_entity_names = entity_df['Entity_name'].iloc[ranked_entity_indices]

    # Get the actual entity name for the current sentence
    actual_entity_name = input_df['entity_name'].iloc[i]
    
    res_data.append(
    {
        'sentence_id': input_df['sentence_id'].iloc[i],
        'linked_entity': ranked_entity_names.iloc[0],
        'actual_entity' : actual_entity_name
    }
    )
    # Step 4: Check if the actual entity is within the top 1, 5, and 10
    if actual_entity_name == ranked_entity_names.iloc[0]:
        top1_correct += 1
    if actual_entity_name in ranked_entity_names.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name in ranked_entity_names.iloc[:10].values:
        top10_correct += 1

# Step 5: Calculate and print ranking accuracies
total_sentences = len(input_df)

top1_accuracy = top1_correct / total_sentences * 100
top5_accuracy = top5_correct / total_sentences * 100
top10_accuracy = top10_correct / total_sentences * 100

print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")
print(f"Top-5 Accuracy: {top5_accuracy:.2f}%")
print(f"Top-10 Accuracy: {top10_accuracy:.2f}%")

Top-1 Accuracy: 60.36%
Top-5 Accuracy: 72.92%
Top-10 Accuracy: 76.69%


In [8]:
ner_df = pd.DataFrame(res_data)
ner_df.to_csv("asr_entity_linked_ImageBIND_results.csv")

In [9]:

# # Process entity audio files in batches
entity_embeddings = process_audio_in_batches(entity_df['Entity_name'].tolist())

# Process sentence audio files in batches
input_embeddings  = process_audio_in_batches(asr_df['ASR_Output'].tolist())



Processing Batches: 100%|██████████| 165/165 [00:50<00:00,  3.25it/s]


In [10]:
from sklearn.metrics.pairwise import cosine_similarity
# Initialize counters for ranking accuracy
top1_correct = 0
top5_correct = 0
top10_correct = 0

res_data = []

# Step 3: Calculate cosine similarity and evaluate rankings
for i, sentence in enumerate(input_df['text']):
    # Get the similarity scores for the current sentence with all entities
    similarity_scores = cosine_similarity(input_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    
    # Get the indices of entities sorted by similarity score (descending order)
    ranked_entity_indices = similarity_scores.argsort()[::-1]
    
    # Get the ranked entity names
    ranked_entity_names = entity_df['Entity_name'].iloc[ranked_entity_indices]

    # Get the actual entity name for the current sentence
    actual_entity_name = input_df['entity_name'].iloc[i]
    
    res_data.append(
    {
        'sentence_id': input_df['sentence_id'].iloc[i],
        'linked_entity': ranked_entity_names.iloc[0],
        'actual_entity' : actual_entity_name
    }
    )
    # Step 4: Check if the actual entity is within the top 1, 5, and 10
    if actual_entity_name == ranked_entity_names.iloc[0]:
        top1_correct += 1
    if actual_entity_name in ranked_entity_names.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name in ranked_entity_names.iloc[:10].values:
        top10_correct += 1

# Step 5: Calculate and print ranking accuracies
total_sentences = len(input_df)

top1_accuracy = top1_correct / total_sentences * 100
top5_accuracy = top5_correct / total_sentences * 100
top10_accuracy = top10_correct / total_sentences * 100

print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")
print(f"Top-5 Accuracy: {top5_accuracy:.2f}%")
print(f"Top-10 Accuracy: {top10_accuracy:.2f}%")

Top-1 Accuracy: 62.83%
Top-5 Accuracy: 76.58%
Top-10 Accuracy: 80.73%


In [11]:
ner_df = pd.DataFrame(res_data)
ner_df.to_csv("asr_entity_linked_ImageBIND_wo_knowledge_results.csv")

In [12]:
# Calculate the substring for the first 20% of each knowledge string
entity_df['Knowledge_20'] = entity_df['Knowledge'].apply(lambda x: x[:int(len(x) * 0.2)])

# # Process entity audio files in batches
entity_embeddings = process_audio_in_batches(entity_df['Knowledge_20'].tolist())

# Process sentence audio files in batches
input_embeddings  = process_audio_in_batches(asr_df['ASR_Output'].tolist())



Processing Batches: 100%|██████████| 165/165 [00:50<00:00,  3.24it/s]


In [13]:
from sklearn.metrics.pairwise import cosine_similarity
# Initialize counters for ranking accuracy
top1_correct = 0
top5_correct = 0
top10_correct = 0

res_data = []

# Step 3: Calculate cosine similarity and evaluate rankings
for i, sentence in enumerate(input_df['text']):
    # Get the similarity scores for the current sentence with all entities
    similarity_scores = cosine_similarity(input_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    
    # Get the indices of entities sorted by similarity score (descending order)
    ranked_entity_indices = similarity_scores.argsort()[::-1]
    
    # Get the ranked entity names
    ranked_entity_names = entity_df['Entity_name'].iloc[ranked_entity_indices]

    # Get the actual entity name for the current sentence
    actual_entity_name = input_df['entity_name'].iloc[i]
    
    res_data.append(
    {
        'sentence_id': input_df['sentence_id'].iloc[i],
        'linked_entity': ranked_entity_names.iloc[0],
        'actual_entity' : actual_entity_name
    }
    )
    # Step 4: Check if the actual entity is within the top 1, 5, and 10
    if actual_entity_name == ranked_entity_names.iloc[0]:
        top1_correct += 1
    if actual_entity_name in ranked_entity_names.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name in ranked_entity_names.iloc[:10].values:
        top10_correct += 1

# Step 5: Calculate and print ranking accuracies
total_sentences = len(input_df)

top1_accuracy = top1_correct / total_sentences * 100
top5_accuracy = top5_correct / total_sentences * 100
top10_accuracy = top10_correct / total_sentences * 100

print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")
print(f"Top-5 Accuracy: {top5_accuracy:.2f}%")
print(f"Top-10 Accuracy: {top10_accuracy:.2f}%")

Top-1 Accuracy: 62.60%
Top-5 Accuracy: 78.75%
Top-10 Accuracy: 84.01%


In [14]:
ner_df = pd.DataFrame(res_data)
ner_df.to_csv("asr_entity_linked_ImageBIND_partial_knowledge_results.csv")